I need to clean this up eventually, I am now back to using 33 landmarks.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from keras.utils import to_categorical
import mediapipe as mp

In [10]:
import sys
sys.path.append("C:\\Users\\Cian\\1 FYP Code\\Code_with_git\\Functions")
from Functions_Squat import detectPose

Import all images that will be used for training. Each class are kept in a single folder.

In [4]:
images = []
O = ['Front','Side R', 'Side L']
for orient in O:
    im = []
    i = 1
    while True:
        path = f"C:\\Users\\Cian\\OneDrive\\Documents\\Imperial\\Year 4\\FYP\\Code and Data\\Orientation\\Training Images\\{orient}\\1 ({i}).jpg"
        image = cv.imread(path)
        # Check if image read successfully
        if image is None:
            break  # Break the loop if no image is read
        im.append(image)
        i += 1  # Increment counter for image name
    images.append(im)
print(len(images))

3


Now extract and normalize landmarks for each image.

In [13]:
def normalise(image):
    # Initialize mediapipe pose class.
    mp_pose = mp.solutions.pose
    
    # Setup the Pose function for images - independently for the images standalone processing.
    pose_image = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.1)
    landmarks = detectPose(image, pose_image)
    XY = np.array([(landmark.x, landmark.y) for landmark in landmarks])
    #only select some landmarks
    mask = np.zeros((33),dtype=bool)
    #now create a list of numbers that represent the coordinates I want to keep
    useful = [11,12,23,24,25,26,27,28]
    for i in useful:
        mask[i]=True
    XY = XY[mask]
    
    #Now normalize the data. Normalize x coord with respect to the maximum and minimum x coord. Then the same for y.
    #Define the maximum and minimum value for x and y coordinates from all landmarks.
    xmax = np.max(XY[:,0])
    xmin = np.min(XY[:,0])
    ymax = np.max(XY[:,1])
    ymin = np.min(XY[:,1])
    
    #normalize the landmarks
    X = (XY[:,0]-xmin)/(xmax-xmin)
    Y = (XY[:,1]-ymin)/(ymax-ymin)
    data = (np.stack((X,Y)).T).reshape(1,16)
    return data

In [ ]:
Data = []

# Loop through each class of images
for i, class_images in enumerate(images):
    norm_landmarks = []
    print(f"Processing class {i}")
    
    # Loop through each individual image
    for j, img in enumerate(class_images):
        try:
            # Extract and normalize all landmarks, then reshape into a vector (ready for the neural net).
            norm_landmarks.append(normalise(img))
            print(f"Processed image {j} in class {i}")
        except Exception as e:
            # Print an error message and continue with the next image.
            print(f"Skipping image {j} in class {i} due to error: {e}")

    Data.append(np.stack(norm_landmarks))

Processing class 0
Processed image 0 in class 0
Processed image 1 in class 0
Processed image 2 in class 0
Processed image 3 in class 0
Processed image 4 in class 0
Processed image 5 in class 0
Processed image 6 in class 0
Processed image 7 in class 0
Processed image 8 in class 0
Processed image 9 in class 0
Processed image 10 in class 0
Processed image 11 in class 0
Processed image 12 in class 0
Processed image 13 in class 0
Processed image 14 in class 0
Processed image 15 in class 0
Processed image 16 in class 0
Processed image 17 in class 0
Processed image 18 in class 0
Processed image 19 in class 0
Processed image 20 in class 0
Processed image 21 in class 0
Processed image 22 in class 0
Processed image 23 in class 0
Processed image 24 in class 0
Processed image 25 in class 0
Processed image 26 in class 0
Processed image 27 in class 0
Processed image 28 in class 0
Processed image 29 in class 0
Processed image 30 in class 0
Processed image 31 in class 0
Processed image 32 in class 0
P

In [6]:
Data = []
Data.append(np.load("C:\\Users\\Cian\\1 FYP Code\\Code_with_git\\Prototype 3\\Camera Angle\\Training Data\\Front.npy"))
Data.append(np.load("C:\\Users\\Cian\\1 FYP Code\\Code_with_git\\Prototype 3\\Camera Angle\\Training Data\\SideR.npy"))
Data.append(np.load("C:\\Users\\Cian\\1 FYP Code\\Code_with_git\\Prototype 3\\Camera Angle\\Training Data\\SideL.npy"))

Now create the Labels.
- [1,0,0] = Forward Facing
- [0,1,0] = Side Right
- [0,0,1] = Side Left

In [13]:
labels = [0]*len(Data[0])+[1]*len(Data[1])+[2]*len(Data[2])
labels = to_categorical(labels,3)

#Concatenate the data into a single array
data = np.concatenate(Data)

#Now shuffle both the data and the labels
perm = np.random.permutation(len(labels))
data = data[perm]
labels = labels[perm]

In [15]:
#Save the labels and the data
np.save("C:\\Users\\Cian\\1 FYP Code\\Code_with_git\\Prototype 3\\Camera Angle\\Training Data\\Data.npy",data)
np.save("C:\\Users\\Cian\\1 FYP Code\\Code_with_git\\Prototype 3\\Camera Angle\\Training Data\\Labels.npy",labels)